<a href="https://colab.research.google.com/github/UF-CIS6930-NLP-Semantalists/semantic-shift-communities/blob/main/cis6930_bert_pre_training_conservative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ga9ok384
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ga9ok384
  Resolved https://github.com/huggingface/transformers to commit fb3aa06cb673d0e2774a2924747d3290135c09cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6928727 sha256=0e0d8a0d9fd21499b84b52cb765f1c957239528160193d48e781fff9b882c99b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3f4uan2f/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190

In [ ]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "bert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> BERT number of parameters: {round(bert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> BERT number of parameters: 110M'
'>>> BERT number of parameters: 110M'


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import torch

text = "This is a [MASK] day."
inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a beautiful day.'
'>>> This is a good day.'
'>>> This is a new day.'
'>>> This is a great day.'
'>>> This is a big day.'


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset

# dataset = pd.read_csv("/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/conservative_base_sample.csv", encoding='utf8')
# full_text = dataset.body
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/Conservative_bert/conservative_base_sample.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3d54ae5d3bb61651/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['body', 'controversiality', 'permalink', 'score', 'subreddit', 'id', 'timestamp', 'preprocess_body', 'num_sentences', 'numWords'],
        num_rows: 194944
    })
})

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["body"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        # result["word_ids"] = result.word_ids()
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=['body','controversiality', 'permalink', 'score', 'subreddit', 'id', 'timestamp', 'preprocess_body', 'num_sentences', 'numWords']
)
tokenized_datasets

Map:   0%|          | 0/194944 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 194944
    })
})

In [ ]:
print(tokenized_datasets['train'][0])

{'input_ids': [101, 2531, 1003, 5993, 1998, 1045, 1005, 1049, 1037, 24247, 1012, 2087, 24247, 2015, 2514, 1996, 2168, 1010, 1998, 2057, 2071, 2191, 3809, 5082, 2065, 2057, 2499, 2362, 1012, 1996, 3160, 2003, 1024, 2054, 5300, 1998, 4896, 5672, 2216, 2029, 4839, 1999, 1996, 3570, 22035, 1029, 2057, 1005, 1040, 3497, 21090, 2045, 1010, 2021, 1045, 2424, 2116, 11992, 2172, 6082, 2000, 2831, 2000, 2055, 2122, 8050, 3314, 2005, 2023, 6635, 3114, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'word_ids': [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, 

In [ ]:
tokenizer.model_max_length

512

In [ ]:
chunk_size = 128

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets['train'][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Post {idx} length: {len(sample)}'")

'>>> Post 0 length: 73'
'>>> Post 1 length: 137'
'>>> Post 2 length: 47'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 257'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 1'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/194944 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 75950
    })
})

In [ ]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"])

"[CLS] 100 % agree and i'm a leftist. most leftists feel the same, and we could make serious progress if we worked together. the question is : what values and institutions replace those which exist in the status quo? we'd likely disagree there, but i find many conservatives much easier to talk to about these fundamental issues for this exact reason. [SEP] [CLS] https : / / www. reddit. com / r / askreddit / comments / db4jdq / what _ is _ your _ absolutely _ favorite _ quote _ youve _ heard / f235mgo / there you are saying you are"

In [ ]:
tokenizer.decode(lm_datasets["train"][0]["labels"])

"[CLS] 100 % agree and i'm a leftist. most leftists feel the same, and we could make serious progress if we worked together. the question is : what values and institutions replace those which exist in the status quo? we'd likely disagree there, but i find many conservatives much easier to talk to about these fundamental issues for this exact reason. [SEP] [CLS] https : / / www. reddit. com / r / askreddit / comments / db4jdq / what _ is _ your _ absolutely _ favorite _ quote _ youve _ heard / f235mgo / there you are saying you are"

In [ ]:
# masking for training
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] 100 % agree and i'[MASK] a leftist. [MASK] leftists [MASK] the same, [MASK] we could make serious progress if we worked together. the question is : what values and institutions replace those which exist [MASK] [MASK] status quo? we'd likely disagree there, but i find many conservatives much easier to [MASK] to about these fundamental issues for this exact reason. [SEP] [CLS] https : / / www. reddit. com / r / [MASK]reddit / comments / db [MASK]jdq / what _ is _ your _ absolutely _ favorite [MASK] quote [MASK] youve _ heard / f235mgo / there you are saying you are'

'>>> shia and https : / / www. reddit. com / r / idiotsincars clark comments / djcygf / [MASK]ᅭ angle _ of _ [MASK] _ [MASK] _ i _ posted _ a _ few _ days _ [MASK] / [MASK]45mr [MASK]o [MASK] here you are pretending to be kuwaiti [MASK]...... so it seems like what you are [MASK] a concern troll. [SEP] [CLS] " we got [MASK] bubble - headed - bleach - blond who comes [MASK] at five she can tell you'bout [MASK] plan

In [ ]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] 100 % agree and [MASK]'m a leftist. most [MASK] [MASK] feel the same, and we [MASK] make serious progress if we worked together. the question is : what values and institutions replace those which exist [MASK] [MASK] [MASK] [MASK] [MASK] we [MASK] d likely disagree there, but i find many conservatives much easier to talk [MASK] about [MASK] fundamental issues for this [MASK] reason. [SEP] [CLS] https [MASK] / [MASK] www [MASK] reddit. com / r / askreddit / comments / db4jdq / [MASK] [MASK] [MASK] _ your [MASK] absolutely _ favorite _ [MASK] [MASK] youve _ heard [MASK] f235mgo [MASK] there you are saying you are'

'>>> [MASK] and https : [MASK] / www [MASK] reddit. com [MASK] r / [MASK] [MASK] [MASK] / comments / [MASK] [MASK] [MASK] / another _ angle _ of _ [MASK] _ idiot _ i _ posted _ a _ few [MASK] days _ [MASK] / f45mr0o / here you are pretending [MASK] be kuwaiti....... [MASK] [MASK] [MASK] like what [MASK] are is a [MASK] [MASK]. [SEP] [CLS] " we got the bubble - heade

In [ ]:
train_size = 74_000
test_size = 1950

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 74000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1950
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-reddit_conservative",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    remove_unused_columns=False,
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=whole_word_masking_data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_conservative into local empty directory.


Download file pytorch_model.bin:   0%|          | 23.4k/418M [00:00<?, ?B/s]

Download file runs/Apr14_03-34-22_c5215eefb2e2/events.out.tfevents.1681444076.c5215eefb2e2.162.2: 100%|#######…

Download file runs/Apr14_03-34-22_c5215eefb2e2/events.out.tfevents.1681443501.c5215eefb2e2.162.0: 100%|#######…

Download file training_args.bin: 100%|##########| 3.56k/3.56k [00:00<?, ?B/s]

...

Clean file runs/Apr14_03-34-22_c5215eefb2e2/events.out.tfevents.1681443501.c5215eefb2e2.162.0:  17%|#7        …

Download file runs/Apr14_03-34-22_c5215eefb2e2/1681443522.1435053/events.out.tfevents.1681443522.c5215eefb2e2.…

Clean file training_args.bin:  28%|##8       | 1.00k/3.56k [00:00<?, ?B/s]

Clean file runs/Apr14_03-34-22_c5215eefb2e2/1681443522.1435053/events.out.tfevents.1681443522.c5215eefb2e2.162…

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [ ]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 75.10


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.340200,3.055798
2,3.122200,2.976186
3,3.068700,2.952759


TrainOutput(global_step=3471, training_loss=3.176950642919032, metrics={'train_runtime': 2221.4667, 'train_samples_per_second': 99.934, 'train_steps_per_second': 1.562, 'total_flos': 1.46078671104e+16, 'train_loss': 3.176950642919032, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 18.98


In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Apr14_23-50-02_af21059ab838/events.out.tfevents.1681519043.af21059ab838.273.2:   0%|         …

Upload file runs/Apr14_23-50-02_af21059ab838/events.out.tfevents.1681516643.af21059ab838.273.0:   0%|         …

To https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_conservative
   7d8e0a3..4fa5256  main -> main

   7d8e0a3..4fa5256  main -> main

To https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_conservative
   4fa5256..0d28ebb  main -> main

   4fa5256..0d28ebb  main -> main



'https://huggingface.co/ChathuriJ/bert-base-uncased-finetuned-reddit_conservative/commit/4fa525618ac06e76649e6f4210c8d131c6bb727b'

# Miscellaneous

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9k9u4q6q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-9k9u4q6q
  Resolved https://github.com/huggingface/transformers to commit 410b61ad7e8f69113a86d0003190e3c392c7c39a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6928596 sha256=db78c3b9af7b93796e88200a3622d839988452381779b5f92237d05653b

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer.is_fast

True

In [ ]:
def batch_iterator():
  for i in range(0, len(data), batch_size):
    yield data[i : i + batch_size]

In [ ]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=52000, max_length=600)

In [ ]:
data[0]

"100% agree and I'm a leftist. Most leftists feel the same, and we could make serious progress if we worked together. The question is: what values and institutions replace those which exist in the status quo?\n\nWe'd likely disagree there, but I find many conservatives much easier to talk to about these fundamental issues for this exact reason."

In [ ]:
new_tokenizer(data[0])

{'input_ids': [2, 2013, 9, 1693, 1068, 49, 11, 53, 41, 2438, 18, 1392, 2666, 1759, 1049, 1468, 16, 1068, 1109, 1405, 1409, 2818, 2678, 1157, 1109, 3128, 3107, 18, 1049, 2056, 1077, 30, 1177, 3562, 1068, 6309, 4975, 1429, 1404, 2373, 1067, 1049, 3773, 6095, 33, 1109, 11, 44, 2133, 2434, 1210, 16, 1136, 49, 1823, 1502, 1910, 1427, 3906, 1060, 2441, 1060, 1199, 1447, 6752, 2570, 1094, 1114, 1911, 1625, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max(data.apply(lambda x: len(new_tokenizer(x)['input_ids'])))

AttributeError: ignored

In [ ]:
%cd /content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/

/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project


In [ ]:
!mkdir Conservative_bert
%cd Conservative_bert/
new_tokenizer.save_pretrained("conservative_bert_tok")

/content/drive/MyDrive/labwork/RA/AMR/CIS 6930 Research Project/Conservative_bert


('conservative_bert_tok/tokenizer_config.json',
 'conservative_bert_tok/special_tokens_map.json',
 'conservative_bert_tok/vocab.txt',
 'conservative_bert_tok/added_tokens.json',
 'conservative_bert_tok/tokenizer.json')

In [ ]:
data = dataset.body
tokenized = data.apply((lambda x: new_tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized

0         [2, 2013, 9, 1693, 1068, 49, 11, 53, 41, 2438,...
1         [2, 1366, 30, 19, 19, 1529, 18, 1701, 18, 1133...
2         [2, 6, 1109, 1552, 1049, 6528, 17, 7656, 17, 8...
3         [2, 1068, 1348, 1104, 155, 59, 2430, 9727, 108...
4         [2, 49, 6878, 1114, 1092, 2635, 1068, 1329, 49...
                                ...                        
194939    [2, 36, 10, 1295, 31, 4107, 4879, 9, 1069, 115...
194940    [2, 2123, 18527, 6645, 3748, 1747, 26896, 3483...
194941    [2, 1257, 1221, 1900, 1213, 1069, 1221, 41, 36...
194942    [2, 1724, 1886, 4672, 1804, 1114, 1307, 1130, ...
194943    [2, 2573, 49, 2236, 3628, 1179, 2637, 1449, 10...
Name: body, Length: 194944, dtype: object

In [ ]:
print(new_tokenizer.tokenize(data[0]))

['100', '%', 'agree', 'and', 'i', "'", 'm', 'a', 'leftist', '.', 'most', 'leftists', 'feel', 'the', 'same', ',', 'and', 'we', 'could', 'make', 'serious', 'progress', 'if', 'we', 'worked', 'together', '.', 'the', 'question', 'is', ':', 'what', 'values', 'and', 'institutions', 'replace', 'those', 'which', 'exist', 'in', 'the', 'status', 'quo', '?', 'we', "'", 'd', 'likely', 'disagree', 'there', ',', 'but', 'i', 'find', 'many', 'conservatives', 'much', 'easier', 'to', 'talk', 'to', 'about', 'these', 'fundamental', 'issues', 'for', 'this', 'exact', 'reason', '.']


In [ ]:
new_tokenizer.decode(new_tokenizer.encode(data[0]))

"[CLS] 100 % agree and i'm a leftist. most leftists feel the same, and we could make serious progress if we worked together. the question is : what values and institutions replace those which exist in the status quo? we'd likely disagree there, but i find many conservatives much easier to talk to about these fundamental issues for this exact reason. [SEP]"

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Apr 13 21:24:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import BertConfig


config = BertConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)